In [1]:
# Importing general libraries
from glob import iglob
import math
import time
import  sys
import difflib
import io
from contextlib import redirect_stdout
import os

# Import ML related libraries
import tensorflow as tf
import keras
from tensorflow.keras.utils import plot_model
from keras.models import load_model
from keras.models import Model, Sequential
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras import backend as K
from keras.initializers import glorot_uniform
#from keras.optimizers import SGD
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score, confusion_matrix, plot_confusion_matrix
#import lime
#import lime.lime_tabular
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

# Checking the version of the of installed / available libraries
# print("Python: " + str(sys.version_info.major) + "." + str(sys.version_info.minor))
# print("Tensorflow: " + str(tf.__version__))
# print("Keras: " + str(keras.__version__))
# print("ScikitLearn: " + str(sklearn.__version__))
# print("Pandas: " + str(pd.__version__))
# print("NumPy: " + str(np.__version__))

# Fetching the data from Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# MetaData
Device_Listing = ['SimpleHome_XCS7_1003_WHT_Security_Camera',  
                 'SimpleHome_XCS7_1002_WHT_Security_Camera',
                 'Provision_PT_838_Security_Camera',
                 'Provision_PT_737E_Security_Camera',
                 'Danmini_Doorbell',
                 'Ennio_Doorbell',
                 'Ecobee_Thermostat',
                 'Samsung_SNH_1011_N_Webcam',
                 'Philips_B120N10_Baby_Monitor']
#Device_Listing = ['Provision_PT_838_Security_Camera', 'Ecobee_Thermostat']


Malware_Listing = ['Mirai','Bashlite']
#Malware_Listing = ['Bashlite']
                   

# Setting Variables


In [ ]:
# Environment Variables

# Known_Devices = ['SimpleHome_XCS7_1002_WHT_Security_Camera']
# Unknown_Devices = ['Philips_B120N10_Baby_Monitor']

# At the moment, it is not expected that both of the following booleans shall be true at the same time. Our analysis is for a single malware at a time.
# Run_For_Malware_Mirai = 1
# Run_For_Malware_Bashlite = 0

# Fisher Score Variables
fisher_score_use_top_n_features = 115 #(max value is 115, reducing this value will reduce the total pool of features to use in training)
use_features_with_fisher_score_greater_than = 0 #(when set to 0, will use all top_n features allowed in above variable. For none zero values, will only use features from the top_n variable above
                                                # whose fisher score is greater than the value set in this variable.)

# This name shall be used to generate relevant save file names at relevant paths
# savefilename='Known-SimpleHomeCamera_Unknown-Provision'
# transfer_savefilename='Transfer-Known-SimpleHomeCamera_Unknown-Provision'


#checkpoint_filepath=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run/original-checkpoint.h5"
#transfer_learning_checkpoint_filepath=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run/transfer-checkpoint.h5"

#results_file_path='/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run/looped_run_results.txt'
#F_Score_File='/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run/F_score_file.txt'


#### Functions

In [ ]:
### Results file writing function

def createAndOpen(filepath, filename, mode):
  os.makedirs(filepath, exist_ok=True)
  return open(filename, mode)

def results_file_writing(data):
  resultsfile = createAndOpen(results_file_path, results_file_name, 'a+')
  resultsfile.write(data)
  resultsfile.close()

def buffer_results_file_writing():
  s = buffer.getvalue()
  resultsfile = createAndOpen(results_file_path, results_file_name, 'a+')
  resultsfile.write(s)
  resultsfile.close() 
  buffer.close()


def tl_results_file_writing(data):
  resultsfile = createAndOpen(tl_results_file_path, tl_results_file_name, 'a+')
  resultsfile.write(data)
  resultsfile.close()

def tl_buffer_results_file_writing():
  s = buffer.getvalue()
  resultsfile = createAndOpen(tl_results_file_path, tl_results_file_name, 'a+')
  resultsfile.write(s)
  resultsfile.close() 
  buffer.close()

### Write Buffer

buffer = io.StringIO()

In [ ]:
def fisher_score(benigndf, attackdf, devicename, F_Score_File):
  scaled_df_fisher = benigndf.append(attackdf.sample(n=benigndf.shape[0]))
  
  #print(scaled_df_fisher.shape)
  scored = [] # declare a list
  indices = {} # declare two dictionaries, purpose unknown
  shps = {}
  classes = ['benign','attack']

  for cl in classes:
      indices[cl] = scaled_df_fisher['class'] == cl # Now here, above created Fisher dataframe shall have two classes only.
                                           # so half of the time the comparison output shall be true and false
                                           # for the rest.
                                           # It creates a Pandas series
      shps[cl] = scaled_df_fisher[indices[cl]].shape[0] # dictionary contains the number of rows of benign traffic and the
                                               # number of rows for the attack traffic

  for col in scaled_df_fisher.columns: # Loop through all the columns of the Fisher DataFrame
      if col == 'class':  # if the column is that of "Class", then skip it.
          continue
      num = 0
      den = 0
      m = scaled_df_fisher[col].mean() # For the column under the current iteration, find the mean value.
                            # no, nothing strange about this. We are finding out the mean of each field/column
                            # in the dataset.
    
      for cl in classes: # For each column (this essentially means that we are considering that column at the moment)
                       # calculate the numerator and the denominator values for the Fisher score per class.
          num += (shps[cl] / scaled_df_fisher.shape[0]) * (m - scaled_df_fisher[indices[cl]][col].mean())**2
                # shps[cl] --> e.g if shps['benign'] then number of rows in it shall be 33032
                # (33032/66064) * ([mean-of-that-whole-column] - mean-of-rows-whose class was cl)**2
          den += (shps[cl] / scaled_df_fisher.shape[0]) * scaled_df_fisher[indices[cl]][col].var()
                # shps[cl] --> e.g if shps['benign'] then number of rows in it shall be 33032
                # (33032/66064) * (variance-of-rows-whose class was cl)
        
      score = {'feature': col, 'score': num / den}
      scored.append(score)
      #print(score)
  scored.sort(key=lambda x: x['score'], reverse=True)
  #scored[:115] #scored is a list of dictionaries    


  # Write down the calculated F-scores in a file.
  with open(F_Score_File, 'a+') as file:
      file.write('\n\n\n============================\n ' + str(devicename) + '\n============================')
      lines = ['\nFeature,Score\n']
      for s in scored:
          lines.append(s['feature'] + ',' + "{0:.2f}".format(s['score']) + '\n')
      file.writelines(lines)
      file.close()

  return scored

In [ ]:
def anomaly_detect_func(device_benign_test, device_attack, known_device, mse_max, quantile_mse_max, model, scaler, fs, confusion_matrix_figure):
  
  results_file_writing('\n\n=======\n Attempting Anomaly detection')
  
  anomaly_detect_01_df_benign = device_benign_test[fs] # Creating Labeled datasets
  anomaly_detect_01_df_benign['malicious'] = 0

  # Removing randomness bias
  # df_malicious = df_known_device_attack.sample(n=df_known_device_benign_test.shape[0], random_state=406)[fs] # Keeping equal parts of benign / malicious data
  anomaly_detect_01_df_malicious = device_attack.sample(n=device_benign_test.shape[0])[fs] # Keeping equal parts of benign / malicious data
  anomaly_detect_01_df_malicious['malicious'] = 1

  anomaly_detect_01_df = anomaly_detect_01_df_benign.append(anomaly_detect_01_df_malicious)
  anomaly_detect_01_X_test = anomaly_detect_01_df.drop(columns=['malicious']).values
  anomaly_detect_01_Y_test = anomaly_detect_01_df['malicious'] # This shall form the ground truth series of values.
  #print(Y_test)

  #for threshold in [mse_max, quantile_mse.max()]:
  for threshold in [mse_max]:
    m = AnomalyModel(model , threshold, scaler)
    anomaly_detect_01_Y_pred = m.predict(anomaly_detect_01_X_test)
    results_file_writing('\nTest-----------------\n')
    #print(f'suggensted threshodl n {best_n}')
    results_file_writing('\nAccuracy: ')
    anomaly_detect_01_acc = accuracy_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    results_file_writing(str(anomaly_detect_01_acc))

    results_file_writing('\nPrecision: ')
    anomaly_detect_01_prec = precision_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    results_file_writing(str(anomaly_detect_01_prec))

    results_file_writing('\nRecall: ')
    anomaly_detect_01_recall = recall_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    results_file_writing(str(anomaly_detect_01_recall))

    results_file_writing('\nF1_Score: ')
    anomaly_detect_01_f1score = f1_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    results_file_writing(str(anomaly_detect_01_f1score))

    results_file_writing('\nCM :')
    anomaly_detect_01_cm = confusion_matrix(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred, labels=[0,1])
    results_file_writing(str(anomaly_detect_01_cm))  

    confusion_matrix_fig=confusion_matrix_figure
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(anomaly_detect_01_cm)
    plt.title('Confusion matrix')
    fig.colorbar(cax)
    labels=['0','1']
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(confusion_matrix_fig)
    plt.show()

  
  return [anomaly_detect_01_acc, anomaly_detect_01_X_test, anomaly_detect_01_Y_test, anomaly_detect_01_prec, anomaly_detect_01_recall, anomaly_detect_01_f1score]

In [ ]:
def best_results_finder(base_acc, mse, quantile_mse, model, scaler, malware, known_device, anomaly_detect_01_X_test, anomaly_detect_01_Y_test, confusion_matrix_figure):

  results_file_writing('\n\n=======\n Attempting to Find the Best Threshold')
  
  #print('Starting Accuracy')
  starting_accuracy = base_acc
  #print(base_acc)

  new_percentile = 0.995
  some_increase_found = False

  best_acc = starting_accuracy
  best_percentile = 0.995

  for test_percentile in [0.995, 0.99, 0.985, 0.98, 0.975, 0.97, 0.965, 0.96, 0.955, 0.95, 0.945, 0.94, 0.935, 0.93, 0.925, 0.92, 0.915, 0.91, 0.905, 0.90]:
    print("Iterating through percentiles to find best threshold")
  
    threshold = mse[mse < mse.quantile(q=test_percentile)].max()
    results_file_writing(str('\nCurrent Percentile: ' + str(test_percentile)))
    results_file_writing(str('\nCurrent Threshold: ' + str(threshold)))
    m = AnomalyModel(model, threshold, scaler)
    anomaly_detect_01_Y_pred = m.predict(anomaly_detect_01_X_test)
    new_acc = accuracy_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    new_precision = precision_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    results_file_writing(str('\n new_acc: ' + str(new_acc) + ' ... new_precision: ' + str(new_precision)))

    if new_acc > best_acc:
      results_file_writing('\n new_acc > best_acc \n\n') 
      best_acc = new_acc
      best_percentile = test_percentile
    else:
      results_file_writing('\n NEW_ACC <= BEST_ACC \n\n')


  #if new_acc > starting_accuracy:
  if best_acc > starting_accuracy:

    results_file_writing('\n\n=======\n Found a Better Quantile, Results are: \n')

    #threshold = mse[mse < mse.quantile(q=previous_percentile)].max()
    threshold = mse[mse < mse.quantile(q=best_percentile)].max()
    #results_file_writing(str('\nCurrent Threshold: ' + str(threshold)))
    m = AnomalyModel(model, threshold, scaler)
    anomaly_detect_01_Y_pred = m.predict(anomaly_detect_01_X_test)

    results_file_writing('\n ------ Results ------\n')

    #results_file_writing(str('\nPercentile: ' + str(previous_percentile)))
    results_file_writing(str('\nPercentile: ' + str(best_percentile)))
    results_file_writing(str('\n Threshold: ' + str(threshold)))
    
    #print(f'suggensted threshodl n {best_n}')
    results_file_writing('\nAccuracy: ')
    anomaly_detect_01_acc = accuracy_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    results_file_writing(str(anomaly_detect_01_acc))

    results_file_writing('\nPrecision: ')
    anomaly_detect_01_prec = precision_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    results_file_writing(str(anomaly_detect_01_prec))

    results_file_writing('\nRecall: ')
    anomaly_detect_01_recall = recall_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    results_file_writing(str(anomaly_detect_01_recall))

    results_file_writing('\nF1_Score: ')
    anomaly_detect_01_f1score = f1_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    results_file_writing(str(anomaly_detect_01_f1score))

    results_file_writing('\nCM :')
    anomaly_detect_01_cm = confusion_matrix(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred, labels=[0,1])
    results_file_writing(str(anomaly_detect_01_cm))  

    confusion_matrix_fig=confusion_matrix_figure
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(anomaly_detect_01_cm)
    plt.title('Confusion matrix')
    fig.colorbar(cax)
    labels=['0','1']
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(confusion_matrix_fig)
    plt.show()

    return [threshold, best_percentile, anomaly_detect_01_acc, anomaly_detect_01_prec, anomaly_detect_01_recall, anomaly_detect_01_f1score]

  else:
    results_file_writing('\n\n=======\n DID NOT find a Better Quantile, Results are: \n')

    threshold = mse[mse < mse.quantile(q=1)].max()
    #results_file_writing(str('\nCurrent Threshold: ' + str(threshold)))
    m = AnomalyModel(model, threshold, scaler)
    anomaly_detect_01_Y_pred = m.predict(anomaly_detect_01_X_test)

    results_file_writing('\n ------ Results ------\n')
    results_file_writing('\nPercentile: 1.0 (global setting)' )

    results_file_writing(str('\n Threshold: ' + str(threshold)))

    #print(f'suggensted threshodl n {best_n}')
    results_file_writing('\nAccuracy: ')
    anomaly_detect_01_acc = accuracy_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    results_file_writing(str(anomaly_detect_01_acc))

    results_file_writing('\nPrecision: ')
    anomaly_detect_01_prec = precision_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    results_file_writing(str(anomaly_detect_01_prec))

    results_file_writing('\nRecall: ')
    anomaly_detect_01_recall = recall_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    results_file_writing(str(anomaly_detect_01_recall))

    results_file_writing('\nF1_Score: ')
    anomaly_detect_01_f1score = f1_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    results_file_writing(str(anomaly_detect_01_f1score))

    results_file_writing('\nCM :')
    anomaly_detect_01_cm = confusion_matrix(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred, labels=[0,1])
    results_file_writing(str(anomaly_detect_01_cm))  
 
    confusion_matrix_fig=confusion_matrix_figure
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(anomaly_detect_01_cm)
    plt.title('Confusion matrix')
    fig.colorbar(cax)
    labels=['0','1']
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(confusion_matrix_fig)    
    plt.show()

    return [threshold, 1.0, anomaly_detect_01_acc, anomaly_detect_01_prec, anomaly_detect_01_recall, anomaly_detect_01_f1score]


'''
  while True:
    print("entering while loop")
    if new_percentile <= 0.9:
      results_file_writing('\nreached quite  low percentile of 0.90 ... Aborting\n')
      anomaly_detect_01_chosen_threshold = quantile_mse.max()
      results_file_writing('\nselected percentile for threshold is 1.0\n')
      results_file_writing(str('\nselected threshold is: ' + str(anomaly_detect_01_chosen_threshold)))    
      break
    threshold = mse[mse < mse.quantile(q=new_percentile)].max()
    results_file_writing(str('\nCurrent Threshold: ' + str(threshold)))
    m = AnomalyModel(model, threshold, scaler)
    anomaly_detect_01_Y_pred = m.predict(anomaly_detect_01_X_test)
    new_acc = accuracy_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    new_precision = precision_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    results_file_writing(str('\n new_acc: ' + str(new_acc) + ' ... new_precision: ' + str(new_precision)))
    if new_acc > base_acc:
      results_file_writing('\n new_acc > base_acc') 
      base_acc = new_acc
      previous_percentile = new_percentile
      new_percentile = new_percentile - 0.005
      results_file_writing(str('\n Next Percentile to test ' + str(new_percentile) + '\n-------------------\n'))
      some_increase_found = True
    elif new_acc < base_acc:
      results_file_writing('\n NEW-ACC < BASE-ACC')
      if some_increase_found:
        results_file_writing('\n New accuracy is lower than previous, stopping... ')
        results_file_writing(str('\n selected percentile for threshold is: ' + str(previous_percentile)))
        results_file_writing('\n selected threshold is: ')
        results_file_writing(str(threshold))
        anomaly_detect_01_chosen_threshold = threshold
        break
      previous_percentile = new_percentile
      new_percentile = new_percentile - 0.005
      results_file_writing(str('\n Next Percentile to test ' + str(new_percentile)))
      continue
    else:
      previous_percentile = new_percentile
      new_percentile = new_percentile - 0.005
      results_file_writing(str('\n Next Percentile to test: ' + str(new_percentile)))
'''



'\n  while True:\n    print("entering while loop")\n    if new_percentile <= 0.9:\n      results_file_writing(\'\nreached quite  low percentile of 0.90 ... Aborting\n\')\n      anomaly_detect_01_chosen_threshold = quantile_mse.max()\n      results_file_writing(\'\nselected percentile for threshold is 1.0\n\')\n      results_file_writing(str(\'\nselected threshold is: \' + str(anomaly_detect_01_chosen_threshold)))    \n      break\n    threshold = mse[mse < mse.quantile(q=new_percentile)].max()\n    results_file_writing(str(\'\nCurrent Threshold: \' + str(threshold)))\n    m = AnomalyModel(model, threshold, scaler)\n    anomaly_detect_01_Y_pred = m.predict(anomaly_detect_01_X_test)\n    new_acc = accuracy_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)\n    new_precision = precision_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)\n    results_file_writing(str(\'\n new_acc: \' + str(new_acc) + \' ... new_precision: \' + str(new_precision)))\n    if new_acc > base_acc:

========================
========================

In [ ]:
def unknown_anomaly_detect_func(device_benign_test, device_attack, known_device, mse_max, model, scaler, fs, confusion_matrix_figure):
  
  tl_results_file_writing('\n\n=======\n Attempting Anomaly detection - Function: Unknown Anomaly Detect')
  
  anomaly_detect_01_df_benign = device_benign_test[fs] # Creating Labeled datasets
  anomaly_detect_01_df_benign['malicious'] = 0

  # Removing randomness bias
  # df_malicious = df_known_device_attack.sample(n=df_known_device_benign_test.shape[0], random_state=406)[fs] # Keeping equal parts of benign / malicious data
  anomaly_detect_01_df_malicious = device_attack.sample(n=device_benign_test.shape[0])[fs] # Keeping equal parts of benign / malicious data
  anomaly_detect_01_df_malicious['malicious'] = 1

  anomaly_detect_01_df = anomaly_detect_01_df_benign.append(anomaly_detect_01_df_malicious)
  anomaly_detect_01_X_test = anomaly_detect_01_df.drop(columns=['malicious']).values
  anomaly_detect_01_Y_test = anomaly_detect_01_df['malicious'] # This shall form the ground truth series of values.
  #print(Y_test)

  #for threshold in [mse_max, quantile_mse.max()]:
  for threshold in [mse_max]:
    m = AnomalyModel(model , threshold, scaler)
    anomaly_detect_01_Y_pred = m.predict(anomaly_detect_01_X_test)
    tl_results_file_writing('\nTest-----------------\n')
    #print(f'suggensted threshodl n {best_n}')
    tl_results_file_writing('\nAccuracy: ')
    anomaly_detect_01_acc = accuracy_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    tl_results_file_writing(str(anomaly_detect_01_acc))

    tl_results_file_writing('\nPrecision: ')
    anomaly_detect_01_prec = precision_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    tl_results_file_writing(str(anomaly_detect_01_prec))

    tl_results_file_writing('\nRecall: ')
    anomaly_detect_01_recall = recall_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    tl_results_file_writing(str(anomaly_detect_01_recall))

    tl_results_file_writing('\nF1_Score: ')
    anomaly_detect_01_f1score = f1_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    tl_results_file_writing(str(anomaly_detect_01_f1score))

    tl_results_file_writing('\nCM :')
    anomaly_detect_01_cm = confusion_matrix(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred, labels=[0,1])
    tl_results_file_writing(str(anomaly_detect_01_cm))  

    confusion_matrix_fig=confusion_matrix_figure
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(anomaly_detect_01_cm)
    plt.title('Confusion matrix')
    fig.colorbar(cax)
    labels=['0','1']
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(confusion_matrix_fig)
    plt.show()

  
  return [anomaly_detect_01_acc, anomaly_detect_01_X_test, anomaly_detect_01_Y_test, anomaly_detect_01_prec, anomaly_detect_01_recall, anomaly_detect_01_f1score]

In [ ]:
def tl_best_results_finder(base_acc, mse, quantile_mse, model, scaler, malware, known_device, anomaly_detect_01_X_test, anomaly_detect_01_Y_test, confusion_matrix_figure):

  tl_results_file_writing('\n\n=======\n Attempting to Find the Best Threshold')
  
  #print('Starting Accuracy')
  starting_accuracy = base_acc
  #print(base_acc)

  new_percentile = 0.995
  some_increase_found = False

  best_acc = starting_accuracy
  best_percentile = 0.995

  for test_percentile in [0.995, 0.99, 0.985, 0.98, 0.975, 0.97, 0.965, 0.96, 0.955, 0.95, 0.945, 0.94, 0.935, 0.93, 0.925, 0.92, 0.915, 0.91, 0.905, 0.90]:
    print("Iterating through percentiles to find best threshold")
  
    threshold = mse[mse < mse.quantile(q=test_percentile)].max()
    tl_results_file_writing(str('\nCurrent Percentile: ' + str(test_percentile)))
    tl_results_file_writing(str('\nCurrent Threshold: ' + str(threshold)))
    m = AnomalyModel(model, threshold, scaler)
    anomaly_detect_01_Y_pred = m.predict(anomaly_detect_01_X_test)
    new_acc = accuracy_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    new_precision = precision_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    tl_results_file_writing(str('\n new_acc: ' + str(new_acc) + ' ... new_precision: ' + str(new_precision)))

    if new_acc > best_acc:
      tl_results_file_writing('\n new_acc > best_acc \n\n') 
      best_acc = new_acc
      best_percentile = test_percentile
    else:
      tl_results_file_writing('\n NEW_ACC <= BEST_ACC \n\n')


  #if new_acc > starting_accuracy:
  if best_acc > starting_accuracy:

    tl_results_file_writing('\n\n=======\n Found a Better Quantile, Results are: \n')

    #threshold = mse[mse < mse.quantile(q=previous_percentile)].max()
    threshold = mse[mse < mse.quantile(q=best_percentile)].max()
    #results_file_writing(str('\nCurrent Threshold: ' + str(threshold)))
    m = AnomalyModel(model, threshold, scaler)
    anomaly_detect_01_Y_pred = m.predict(anomaly_detect_01_X_test)

    tl_results_file_writing('\n ------ Results ------\n')

    #tl_results_file_writing(str('\nPercentile: ' + str(previous_percentile)))
    tl_results_file_writing(str('\nPercentile: ' + str(best_percentile)))
    tl_results_file_writing(str('\n Threshold: ' + str(threshold)))
    
    #print(f'suggensted threshodl n {best_n}')
    tl_results_file_writing('\nAccuracy: ')
    anomaly_detect_01_acc = accuracy_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    tl_results_file_writing(str(anomaly_detect_01_acc))

    tl_results_file_writing('\nPrecision: ')
    anomaly_detect_01_prec = precision_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    tl_results_file_writing(str(anomaly_detect_01_prec))

    tl_results_file_writing('\nRecall: ')
    anomaly_detect_01_recall = recall_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    tl_results_file_writing(str(anomaly_detect_01_recall))

    tl_results_file_writing('\nF1_Score: ')
    anomaly_detect_01_f1score = f1_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    tl_results_file_writing(str(anomaly_detect_01_f1score))

    tl_results_file_writing('\nCM :')
    anomaly_detect_01_cm = confusion_matrix(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred, labels=[0,1])
    tl_results_file_writing(str(anomaly_detect_01_cm))  

    confusion_matrix_fig=confusion_matrix_figure
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(anomaly_detect_01_cm)
    plt.title('Confusion matrix')
    fig.colorbar(cax)
    labels=['0','1']
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(confusion_matrix_fig)
    plt.show()

    return [threshold, best_percentile, anomaly_detect_01_acc, anomaly_detect_01_prec, anomaly_detect_01_recall, anomaly_detect_01_f1score]

  else:
    tl_results_file_writing('\n\n=======\n DID NOT find a Better Quantile, Results are: \n')

    threshold = mse[mse < mse.quantile(q=1)].max()
    #results_file_writing(str('\nCurrent Threshold: ' + str(threshold)))
    m = AnomalyModel(model, threshold, scaler)
    anomaly_detect_01_Y_pred = m.predict(anomaly_detect_01_X_test)

    tl_results_file_writing('\n ------ Results ------\n')
    tl_results_file_writing('\nPercentile: 1.0 (global setting)' )

    tl_results_file_writing(str('\n Threshold: ' + str(threshold)))

    #print(f'suggensted threshodl n {best_n}')
    tl_results_file_writing('\nAccuracy: ')
    anomaly_detect_01_acc = accuracy_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    tl_results_file_writing(str(anomaly_detect_01_acc))

    tl_results_file_writing('\nPrecision: ')
    anomaly_detect_01_prec = precision_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    tl_results_file_writing(str(anomaly_detect_01_prec))

    tl_results_file_writing('\nRecall: ')
    anomaly_detect_01_recall = recall_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    tl_results_file_writing(str(anomaly_detect_01_recall))

    tl_results_file_writing('\nF1_Score: ')
    anomaly_detect_01_f1score = f1_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    tl_results_file_writing(str(anomaly_detect_01_f1score))

    tl_results_file_writing('\nCM :')
    anomaly_detect_01_cm = confusion_matrix(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred, labels=[0,1])
    tl_results_file_writing(str(anomaly_detect_01_cm))  
 
    confusion_matrix_fig=confusion_matrix_figure
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(anomaly_detect_01_cm)
    plt.title('Confusion matrix')
    fig.colorbar(cax)
    labels=['0','1']
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(confusion_matrix_fig)    
    plt.show()

    return [threshold, 1.0, anomaly_detect_01_acc, anomaly_detect_01_prec, anomaly_detect_01_recall, anomaly_detect_01_f1score]

'''
  while True:
    print("entering while loop")
    if new_percentile <= 0.9:
      tl_results_file_writing('\nreached quite  low percentile of 0.90 ... Aborting\n')
      anomaly_detect_01_chosen_threshold = quantile_mse.max()
      tl_results_file_writing('\nselected percentile for threshold is 1.0\n')
      tl_results_file_writing(str('\nselected threshold is: ' + str(anomaly_detect_01_chosen_threshold)))    
      break
    threshold = mse[mse < mse.quantile(q=new_percentile)].max()
    tl_results_file_writing(str('\nCurrent Threshold: ' + str(threshold)))
    m = AnomalyModel(model, threshold, scaler)
    anomaly_detect_01_Y_pred = m.predict(anomaly_detect_01_X_test)
    new_acc = accuracy_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    new_precision = precision_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)
    tl_results_file_writing(str('\n new_acc: ' + str(new_acc) + ' ... new_precision: ' + str(new_precision)))
    if new_acc > base_acc:
      tl_results_file_writing('\n new_acc > base_acc') 
      base_acc = new_acc
      previous_percentile = new_percentile
      new_percentile = new_percentile - 0.005
      tl_results_file_writing(str('\n Next Percentile to test ' + str(new_percentile) + '\n-------------------\n'))
      some_increase_found = True
    elif new_acc < base_acc:
      tl_results_file_writing('\n NEW-ACC < BASE-ACC')
      if some_increase_found:
        tl_results_file_writing('\n New accuracy is lower than previous, stopping... ')
        tl_results_file_writing(str('\n selected percentile for threshold is: ' + str(previous_percentile)))
        tl_results_file_writing('\n selected threshold is: ')
        tl_results_file_writing(str(threshold))
        anomaly_detect_01_chosen_threshold = threshold
        break
      previous_percentile = new_percentile
      new_percentile = new_percentile - 0.005
      tl_results_file_writing(str('\n Next Percentile to test ' + str(new_percentile)))
      continue
    else:
      previous_percentile = new_percentile
      new_percentile = new_percentile - 0.005
      tl_results_file_writing(str('\n Next Percentile to test: ' + str(new_percentile)))


'''


'\n  while True:\n    print("entering while loop")\n    if new_percentile <= 0.9:\n      tl_results_file_writing(\'\nreached quite  low percentile of 0.90 ... Aborting\n\')\n      anomaly_detect_01_chosen_threshold = quantile_mse.max()\n      tl_results_file_writing(\'\nselected percentile for threshold is 1.0\n\')\n      tl_results_file_writing(str(\'\nselected threshold is: \' + str(anomaly_detect_01_chosen_threshold)))    \n      break\n    threshold = mse[mse < mse.quantile(q=new_percentile)].max()\n    tl_results_file_writing(str(\'\nCurrent Threshold: \' + str(threshold)))\n    m = AnomalyModel(model, threshold, scaler)\n    anomaly_detect_01_Y_pred = m.predict(anomaly_detect_01_X_test)\n    new_acc = accuracy_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)\n    new_precision = precision_score(anomaly_detect_01_Y_test, anomaly_detect_01_Y_pred)\n    tl_results_file_writing(str(\'\n new_acc: \' + str(new_acc) + \' ... new_precision: \' + str(new_precision)))\n    if new_

#### AutoEncoder Model

In [ ]:
# Now creating a ____________ model
def create_model(input_dim):
    inp = Input(shape=(input_dim,))
    encoder = Dense(int(math.ceil(0.80 * input_dim)), activation="tanh")(inp)
    encoder = Dense(int(math.ceil(0.60 * input_dim)), activation="tanh")(encoder)
    encoder = Dense(int(math.ceil(0.40 * input_dim)), activation="tanh")(encoder)
    encoder = Dense(int(math.ceil(0.20 * input_dim)), activation="tanh")(encoder)
    decoder = Dense(int(math.ceil(0.40 * input_dim)), activation="tanh")(encoder)
    decoder = Dense(int(math.ceil(0.60 * input_dim)), activation="tanh")(decoder)
    decoder = Dense(int(math.ceil(0.80 * input_dim)), activation="tanh")(decoder)
    decoder = Dense(input_dim)(decoder)
    return Model(inp, decoder)

#### Anomaly Detection Class

In [ ]:
 #A Class definition for executing predictions
class AnomalyModel:
    def __init__(self, model, threshold, scaler):
        self.model = model
        self.threshold = threshold
        self.scaler = scaler

    def predict(self, x):
        x_pred = self.model.predict(x)
        mse = np.mean(np.power(x - x_pred, 2), axis=1)
        y_pred = mse > self.threshold
        return y_pred.astype(int)

## Making a Giant Loop

In [ ]:
for malware in Malware_Listing: # iterate for each malware.

  
  for known_device in Device_Listing:

    ### Defining file location variables

    checkpoint_filepath=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}/original-model/original-checkpoint.h5"
    results_file_path=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}"
    results_file_name=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}/looped_run_results.txt"
    Known_F_Score_File=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}/F_score_file.txt"
    
    known_device_confusion_matrix_figure_percentile_1=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}/original-model/anomaly-detect-CM-percentile-1.png"
    known_device_confusion_matrix_figure_best_percentile=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}/original-model/anomaly-detect-CM-percentile-best.png"
    
    results_file_writing(str('\n\n\n ==== \n LOOPING FOR MALWARE: '  + str(malware) + '\n'))
    results_file_writing('KNOWN DEVICE: ' + str(known_device) + '\n')

    ### Gathering Benign data of the known device

    filepaths = []
    print(known_device)
    benign_path = f"/content/drive/My Drive/thesis-unsub/data/{known_device}/benign_traffic.csv"
    filepaths.append(benign_path)

    df_known_device_benign = pd.concat((pd.read_csv(f) for f in filepaths), ignore_index=True)
    results_file_writing("\n Benign DataFrame Information\n------")
    
    buffer = io.StringIO()
    df_known_device_benign.info(buf=buffer)
    buffer_results_file_writing()
    results_file_writing('\n\n')

    ### Gathering Attack data of the known device

    if malware == "Mirai":
      
      if known_device == "Ennio_Doorbell":
        continue
      if known_device == "Samsung_SNH_1011_N_Webcam":
        continue

      results_file_writing("\n\n LOADING ATTACK DATA OF KNOWN DEVICE AGAINST MALWARE: MIRAI\n")
      
      Mirai_Filepaths = []
      attack_path = f"/content/drive/My Drive/thesis-unsub/data/{known_device}/mirai_attacks/"
        
      for files in ['ack.csv', 'scan.csv', 'syn.csv', 'udp.csv', 'udpplain.csv']:
        filepath = attack_path + str(files)
        Mirai_Filepaths.append(filepath)

      results_file_writing(str(Mirai_Filepaths))

      df_known_device_attack= pd.concat((pd.read_csv(f) for f in Mirai_Filepaths), ignore_index=True)
      
      results_file_writing("\n Attack DataFrame Information\n------")
      buffer = io.StringIO()
      df_known_device_attack.info(buf=buffer)
      buffer_results_file_writing()
      results_file_writing('\n\n')    

    elif malware == "Bashlite":
      results_file_writing("\n\n LOADING ATTACK DATA OF KNOWN DEVICE AGAINST MALWARE: BASHLITE\n")

      Bashlite_Filepaths = []
      
      attack_path = f"/content/drive/My Drive/thesis-unsub/data/{known_device}/gafgyt_attacks/"
      
      for files in ['combo.csv', 'junk.csv', 'scan.csv', 'tcp.csv', 'udp.csv']:
        filepath = attack_path + str(files)
        Bashlite_Filepaths.append(filepath)

      results_file_writing(str(Bashlite_Filepaths))

      df_known_device_attack = pd.concat((pd.read_csv(f) for f in Bashlite_Filepaths), ignore_index=True)
      results_file_writing("\n Attack DataFrame Information\n------")
      
      buffer = io.StringIO()
      df_known_device_attack.info(buf=buffer)
      buffer_results_file_writing()
      results_file_writing('\n\n')


      ### Splitting the benign dataset of the Known dataset.

    results_file_writing("\n\n\n Splitting Benign Data into 3 parts for training, optimization and test in a 60/20/20 split\n")

    df_known_device_benign_train, df_known_device_benign_opt, df_known_device_benign_test = np.split(df_known_device_benign.sample(frac=1), [int(3/5*len(df_known_device_benign)), int(4/5*len(df_known_device_benign))])

    results_file_writing("Split Information\n")
    results_file_writing("df_known_device_benign_train: " + str(df_known_device_benign_train.shape) + "\n")
    results_file_writing("df_known_device_benign_opt:   " + str(df_known_device_benign_opt.shape) + "\n")
    results_file_writing("df_known_device_benign_test:  " + str(df_known_device_benign_test.shape) + "\n")


    results_file_writing("\n\n Information about the Split\nTraining Data\n---")
    buffer = io.StringIO()
    df_known_device_benign_train.info(buf=buffer)
    buffer_results_file_writing()

    results_file_writing("\n Optimization Data\n---")
    buffer = io.StringIO()
    df_known_device_benign_opt.info(buf=buffer)
    buffer_results_file_writing()

    results_file_writing("\n Test Data \n---")
    buffer = io.StringIO()
    df_known_device_benign_test.info(buf=buffer)
    buffer_results_file_writing() 

    ### Scaling the bengin and attack datasets of the known device.
    col_list = []
    for col in df_known_device_benign_train.columns:
      col_list.append(col)

    scaler = MinMaxScaler()
    scaled_df_known_device_benign_train = scaler.fit_transform(df_known_device_benign_train)
    scaled_df_known_device_benign_opt = scaler.transform(df_known_device_benign_opt)
    scaled_df_known_device_benign_test = scaler.transform(df_known_device_benign_test)
    scaled_df_known_device_attack = scaler.transform(df_known_device_attack)
    #Converting NumPy arrays back to Pandas DataFrames
    scaled_df_known_device_benign_train = pd.DataFrame(scaled_df_known_device_benign_train, columns=col_list)
    scaled_df_known_device_benign_opt = pd.DataFrame(scaled_df_known_device_benign_opt, columns=col_list)
    scaled_df_known_device_benign_test = pd.DataFrame(scaled_df_known_device_benign_test, columns=col_list)
    scaled_df_known_device_attack = pd.DataFrame(scaled_df_known_device_attack, columns=col_list)    


    ### Labelling the data.
    scaled_df_known_device_attack['class'] = 'attack'
    scaled_df_known_device_benign_train['class'] = 'benign'
    scaled_df_known_device_benign_opt['class'] = 'benign'
    scaled_df_known_device_benign_test['class'] = 'benign'    

    ### FisherScoring Function
    scores = fisher_score(scaled_df_known_device_benign_train, scaled_df_known_device_attack, known_device, Known_F_Score_File)

    results_file_writing('\n\n\n --> Calculated the Fisher Score for known Device, see the F_Score_file')

    ### Model Training Time
    results_file_writing('\n\n\n --> Starting AutoEncoder Model Training')
    for top_n_features in [fisher_score_use_top_n_features]:
      fs = []
      for it in scores[:top_n_features]:
        if it['score'] > use_features_with_fisher_score_greater_than:  
          fs.append(it['feature'])    

      features_to_be_used = len(fs)
      DF_BENIGN_TRAIN = scaled_df_known_device_benign_train[fs]
      DF_BENIGN_OPT = scaled_df_known_device_benign_opt[fs]
      DF_BENIGN_TEST = scaled_df_known_device_benign_test[fs]

      model = create_model(len(fs))
      model.compile(loss="mean_squared_error",
                    optimizer="adam")
    
      Initial_Weights = model.get_weights()

      cp = ModelCheckpoint(filepath=checkpoint_filepath,
                                  monitor='val_loss',
                               save_best_only=True,
                               verbose=1)
      es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
      start = time.time()
      epochs = 100
      history = model.fit(DF_BENIGN_TRAIN, DF_BENIGN_TRAIN,
                    epochs=epochs,
                    validation_data=(DF_BENIGN_OPT, DF_BENIGN_OPT),
                    verbose=1,
                    callbacks=[cp, es])
      end = time.time()
      results_file_writing('\n\n ----> Model Training Time Is: ')
      results_file_writing(str(end - start))
      results_file_writing('\n\n')

      with open(results_file_name, 'at') as f:
        with redirect_stdout(f):
          model.summary()
      f.close()

      original_model_path=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}/original-model/model.h5"
      model.save(original_model_path)

    original_model_plot_path=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}/original-model/training-plot.png"
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Opt'], loc='upper left')
    plt.savefig(original_model_plot_path)    
    #plt.show()

    ### Defining Anomaly Error Threshold

    results_file_writing('\n\n  <<  Defining Anomaly Threshold  >>  \n\n')
    
    df_benign_known_device_test_predictions = model.predict(scaled_df_known_device_benign_test[fs])
    difference = scaled_df_known_device_benign_test[fs] - df_benign_known_device_test_predictions
    
    mse = np.mean(np.power(scaled_df_known_device_benign_test[fs] - df_benign_known_device_test_predictions, 2), axis=1) # Mean-Squared-Error is calculated for each row,
    mean = mse.mean() # We calculate the mean of the whole series, that is the mean mse for all the benign test cases.
    std = mse.std() # And we calculate the standard deviation of the mse series, containing only the test cases.

    mse = pd.Series(mse) #converting np array to dataframe series

    results_file_writing(str("MIN of MSE: " + str(mse.min())) + "\n")
    results_file_writing(str("MAX of MSE: " + str(mse.max())) + "\n")
    results_file_writing(str("MEAN of MSE: " + str(mean)) + "\n")
    results_file_writing(str("STD of MSE:  " + str(std)) + "\n")

    mse_min = mse.min()
    mse_max = mse.max()
    results_file_writing(str("\nMax Threshold = MSE_MAX: " + str(mse_max))  + "\n")

    max_std_multiplier = int((mse_max - mean) / std)
    results_file_writing(str("\nMAX_STD_Multiplier = " + str(max_std_multiplier)) + "\n")

    original_model_error_distribution=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}/original-model/error-distribution.png"
    mse_plot = mse.plot.hist(bins=20, color='#0033cc')
    mse_plot.set_facecolor('#ffffff')
    mse_plot.set_xlim((mse_min, mse_max))
    mse_plot.set_ylim((0, mse[mse <= mse_max].count()))
    fig = mse_plot.get_figure()
    fig.savefig(original_model_error_distribution)

    
    ### Doing the quantile MSE stuff
    results_file_writing('\n\n --> Doing the quantile MSE stuff\n')

    test_percentile = 1
    quantile_mse = mse[mse <= mse.quantile(q=test_percentile)]  # <= is important here

    results_file_writing(str("MIN of QUANTILE_MSE: " + str(quantile_mse.min())))
    results_file_writing(str("MAX of QUANTILE_MSE: " + str(quantile_mse.max())))
    results_file_writing(str("MEAN of QUANTILE_MSE: " + str(quantile_mse.mean())))
    results_file_writing(str("STD of QUNATILE_MSE:  " + str(quantile_mse.std())))
    results_file_writing(str("Size of Series below 95th quantile: " + str(quantile_mse.size)))
    results_file_writing(str("\nSuggested Minimum Threshold = QUANTILE_MSE_MAX: " + str(quantile_mse.max())))

    min_std_multiplier = int((quantile_mse.max() - quantile_mse.mean()) / quantile_mse.std())
    results_file_writing(str("\nMIN_STD_Multiplier = " + str(min_std_multiplier)))

    original_model_quantile_error_distribution=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}/original-model/quantile-error-distribution.png"
    quantile_mse_plot = quantile_mse.plot.hist(bins=20, color='#0033cc')
    quantile_mse_plot.set_facecolor('#ffffff')
    quantile_mse_plot.set_xlim((quantile_mse.min(), quantile_mse.max()))
    quantile_mse_plot.set_ylim((0, quantile_mse[quantile_mse <= quantile_mse.max()].count()))
    fig = mse_plot.get_figure()
    fig.savefig(original_model_quantile_error_distribution)


    ### Getting a base accuracy of the model using MSE-MAX threshold.

    results = anomaly_detect_func(scaled_df_known_device_benign_test, scaled_df_known_device_attack, known_device, mse_max, quantile_mse.max(), model, scaler, fs, known_device_confusion_matrix_figure_percentile_1)
    base_acc = results[0]    
    anomaly_detect_01_X_test = results[1]
    anomaly_detect_01_Y_test  = results[2]



    ### Finding the threshold with best scores

    results_file_writing('\n\n <<--=== Finding Best Performning Threshold ===-->> \n')

    original_model_best_results = best_results_finder(base_acc, mse, quantile_mse, model, scaler, malware, known_device, anomaly_detect_01_X_test, anomaly_detect_01_Y_test, known_device_confusion_matrix_figure_best_percentile)
    original_model_best_threshold = original_model_best_results[0]
    original_model_best_percentile = original_model_best_results[1]
    original_model_best_accuracy = original_model_best_results[2]
    original_model_best_precision = original_model_best_results[3]
    original_model_best_recall = original_model_best_results[4]
    original_model_best_f1score = original_model_best_results[5]


    ### 
    results_file_writing('\n\n\n ===-->>> KNOWN DEVICE - ORIGINAL MODEL PERFORMANCE ')
    results_file_writing(str(known_device))
    results_file_writing('\n')
    results_file_writing(str('ACCURACY: ' + str(original_model_best_accuracy)))
    results_file_writing('\n')
    results_file_writing(str('PRECISION: ' + str(original_model_best_precision)))
    results_file_writing('\n')
    results_file_writing(str('RECALL: ' + str(original_model_best_recall)))
    results_file_writing('\n')
    results_file_writing(str('F1-SCORE: ' + str(original_model_best_f1score)))
    results_file_writing('\n ------- \n')

    results_file_writing('\n\n\n ===-->>> UNKNOWN DEVICE - TRANSFER MODEL PERFORMANCE ')




    ## <<<-- ============= TIME TO DO THE UNKOWN DEVICE ==========-->>>





    for unknown_device in Device_Listing:

      if known_device == unknown_device:
        print("unknown device is same as known device")
        continue
      

      ### Defining file location variables

      transfer_learning_checkpoint_filepath=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}/{unknown_device}/transfer-checkpoint.h5"
      tl_results_file_path=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}/{unknown_device}"
      tl_results_file_name=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}/{unknown_device}/looped_run_results.txt"
      Unknown_F_Score_File=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}/{unknown_device}/Unknown_F_score_file.txt"    
      unknown_device_confusion_matrix_figure_original_model_threshold=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}/{unknown_device}/anomaly-detect-CM-originalModel-threshold.png"
      unknown_device_confusion_matrix_figure_transfer_model_threshold=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}/{unknown_device}/anomaly-detect-CM-TransferModel-threshold.png"
      unknown_device_confusion_matrix_figure_transfer_model_best_percentile=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}/{unknown_device}/anomaly-detect-CM-TransferModel-BestPercentile.png"
      unknown_device_confusion_matrix_figure_transfer_model_final_performance=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}/{unknown_device}/anomaly-detect-CM-TransferModel-FinalPerformance.png"

      tl_results_file_writing(str('\n\n\n ==== \n TRANSFER LEARINING  \n ====: '  + str(malware) + '\n'))
      tl_results_file_writing('UNKNOWN DEVICE: ' + str(unknown_device) + '\n')

      ### Gathering Benign data of the UNknown device

      filepaths = []
      print(unknown_device)
      benign_path = f"/content/drive/My Drive/thesis-unsub/data/{unknown_device}/benign_traffic.csv"
      filepaths.append(benign_path)

      df_unknown_device_benign = pd.concat((pd.read_csv(f) for f in filepaths), ignore_index=True)
      tl_results_file_writing("\n Benign DataFrame Information\n------")
    
      buffer = io.StringIO()
      df_unknown_device_benign.info(buf=buffer)
      tl_buffer_results_file_writing()
      tl_results_file_writing('\n\n')

      ### Gathering Attack data of the known device

      if malware == "Mirai":
    
        if unknown_device == "Ennio_Doorbell":
          continue
        if unknown_device == "Samsung_SNH_1011_N_Webcam":
          continue

        tl_results_file_writing("\n\n LOADING ATTACK DATA OF UNKNOWN DEVICE AGAINST MALWARE: MIRAI\n")
      
        Mirai_Filepaths = []
        attack_path = f"/content/drive/My Drive/thesis-unsub/data/{unknown_device}/mirai_attacks/"
       
        for files in ['ack.csv', 'scan.csv', 'syn.csv', 'udp.csv', 'udpplain.csv']:
          filepath = attack_path + str(files)
          Mirai_Filepaths.append(filepath)

        tl_results_file_writing(str(Mirai_Filepaths))

        df_unknown_device_attack = pd.concat((pd.read_csv(f) for f in Mirai_Filepaths), ignore_index=True)
      
        tl_results_file_writing('\n\n\n Attack DataFrame Information\n------')
        buffer = io.StringIO()
        df_unknown_device_attack.info(buf=buffer)
        tl_buffer_results_file_writing()
        tl_results_file_writing('\n\n')    

      elif malware == "Bashlite":
        tl_results_file_writing("\n\n LOADING ATTACK DATA OF KNOWN DEVICE AGAINST MALWARE: BASHLITE\n")

        Bashlite_Filepaths = []
      
        attack_path = f"/content/drive/My Drive/thesis-unsub/data/{unknown_device}/gafgyt_attacks/"
    
        for files in ['combo.csv', 'junk.csv', 'scan.csv', 'tcp.csv', 'udp.csv']:
          filepath = attack_path + str(files)
          Bashlite_Filepaths.append(filepath)

        tl_results_file_writing(str(Bashlite_Filepaths))

        df_unknown_device_attack = pd.concat((pd.read_csv(f) for f in Bashlite_Filepaths), ignore_index=True)
        tl_results_file_writing("\n Attack DataFrame Information\n------")
      
        buffer = io.StringIO()
        df_unknown_device_attack.info(buf=buffer)
        tl_buffer_results_file_writing()
        tl_results_file_writing('\n\n')


      ### Scaling the datasets on unknown devices
      scaled_df_unknown_device_benign = scaler.transform(df_unknown_device_benign)
      scaled_df_unknown_device_attack = scaler.transform(df_unknown_device_attack)
      #Converting NumPy arrays back to Pandas DataFrames
      scaled_df_unknown_device_benign = pd.DataFrame(scaled_df_unknown_device_benign, columns=col_list)
      scaled_df_unknown_device_attack = pd.DataFrame(scaled_df_unknown_device_attack, columns=col_list)

      scaled_df_unknown_device_attack['class'] = 'attack'
      scaled_df_unknown_device_benign['class'] = 'benign'


      #### Sampling the dataset for training, rest shall remain unseen to the model
      # sampling some data from the dataset. We shall sample this data to check if it improves the autoencoder accuracy.
      # and splitting this data into three parts, for training, optimization and validation.

      length_unknown_device_benign = len(scaled_df_unknown_device_benign)

      # extracting a small sample for re-training the model
      scaled_df_unknown_device_benign_sample, scaled_df_unknown_device_benign_remaining = np.split(scaled_df_unknown_device_benign.sample(frac=1), [int(0.10*length_unknown_device_benign)])

      # splitting the small sample of data into train, opt and test datasets.
      scaled_df_unknown_device_benign_sample_train, scaled_df_unknown_device_benign_sample_opt, scaled_df_unknown_device_benign_sample_test = np.split(scaled_df_unknown_device_benign_sample, [int(3/5*len(scaled_df_unknown_device_benign_sample)), int(4/5*len(scaled_df_unknown_device_benign_sample))])

      tl_results_file_writing('\n\nINFORMATION ON SAMPLED DATA')
      buffer = io.StringIO()
      scaled_df_unknown_device_benign_sample.info(buf=buffer)
      tl_buffer_results_file_writing()

      tl_results_file_writing("\nINFORMATION ON REMAINING BENIGN DATA")
      buffer = io.StringIO()
      scaled_df_unknown_device_benign_remaining.info(buf=buffer)
      tl_buffer_results_file_writing()

      tl_results_file_writing("\nSAMPLE DATA SPLIT FOR TRAIN, OPT AND TEST")
      buffer = io.StringIO()
      scaled_df_unknown_device_benign_sample_train.info(buf=buffer)
      tl_buffer_results_file_writing()

      buffer = io.StringIO()
      scaled_df_unknown_device_benign_sample_opt.info(buf=buffer)
      tl_buffer_results_file_writing

      buffer = io.StringIO()
      scaled_df_unknown_device_benign_sample_test.info(buf=buffer)
      tl_buffer_results_file_writing


      ## Skipping the Fisher score part
      #unknown_scored = fisher_score(scaled_df_unknown_device_benign_sample, scaled_df_unknown_device_attack, unknown_device, Unknown_F_Score_File)
      
      #print(unknown_scored)

      ### Checking the Original Anomaly models performance against Unknown sampled data.
      tl_results_file_writing('\n\n Model Accuracy for Malware against unknown device')
      pre_tl_results = unknown_anomaly_detect_func(scaled_df_unknown_device_benign_sample, scaled_df_unknown_device_attack, unknown_device, original_model_best_threshold, model, scaler, fs, unknown_device_confusion_matrix_figure_original_model_threshold)
      pre_tl_base_acc = pre_tl_results[0]  



      ##### TRANSFERING LEARNING ####


      tl_results_file_writing("\n\n CREATING A MODEL FOR TRANSFER LEARNING")
      # making a copy of the original model

      transfer_model = model

      # Freezing all layers except the three inner most layers.

      for layer in transfer_model.layers[0:3]:
        layer.trainable = False

      for layer in transfer_model.layers[-3:]:
        layer.trainable = False

      for layer in transfer_model.layers[:]:
        print(layer, layer.trainable)

      ## Re-Initialization of layers in the center
      initializer = keras.initializers.GlorotUniform()
      #s = K.get_session()
      for layer in transfer_model.layers[3:6]:
        #layer.kernel.initializer.run(session=s)
        layer.set_weights([initializer(shape=w.shape) for w in layer.get_weights()])

      tl_results_file_writing('\n we need to be careful on using the same number of features on which the earlier model is built \n')

      tl_results_file_writing(str(len(fs)))
      tl_results_file_writing('\n')
      tl_results_file_writing(str(transfer_model.layers[0].input_shape[0][1]))
      tl_results_file_writing('\n')

      # Here cap the max number of features to be used o be equal to the input layer of the previous model.

      max_features = transfer_model.layers[0].input_shape[0][1]
      print("Max Features: " + str(max_features))
      for top_n_features in [max_features]: # for loop runs only once. because there is a single item in the list only. The number
                                            # describes the number of top features that shall be selected 
                                            # fs = [it['feature'] for it in scored[:top_n_features]] # fs is a list of feature names in order of decreasing
                                            # F-score
        print("Top N Features" + str(top_n_features))
        unknown_fs = []
        for it in scores[:top_n_features]:
          #if it['score'] > use_features_with_fisher_score_greater_than:  
          unknown_fs.append(it['feature'])

        features_to_be_used = len(unknown_fs)
        tl_results_file_writing(str(features_to_be_used))
        tl_results_file_writing('\n')

        DF_UNKNOWN_BENIGN_TRAIN = scaled_df_unknown_device_benign_sample_train[unknown_fs]
        DF_UNKNOWN_BENIGN_OPT = scaled_df_unknown_device_benign_sample_opt[unknown_fs]
        DF_UNKNOWN_BENIGN_TEST = scaled_df_unknown_device_benign_sample_test[unknown_fs]
    
        #checkpoint_filepath=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/Generic_Anomaly_Detection_Malware_TRANSFER-LEARNING-SimpleHomeCamera-{features_to_be_used}.h5"
        transfer_cp = ModelCheckpoint(filepath=transfer_learning_checkpoint_filepath,
                                  monitor='val_loss',
                               save_best_only=True,
                               verbose=1)
    
        transfer_es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        start = time.time()
        epochs = 100
        transfer_history = transfer_model.fit(DF_UNKNOWN_BENIGN_TRAIN, DF_UNKNOWN_BENIGN_TRAIN,
                    epochs=epochs,
                    validation_data=(DF_UNKNOWN_BENIGN_OPT, DF_UNKNOWN_BENIGN_OPT),
                    verbose=1,
                    callbacks=[transfer_cp, transfer_es])
    
        end = time.time()
        tl_results_file_writing('\n\n ----> Transfer Model Training Time Is: ')
        tl_results_file_writing(str(end - start))
        tl_results_file_writing('\n\n')

        with open(tl_results_file_name, 'at') as f:
          with redirect_stdout(f):
            transfer_model.summary()
        f.close()

        tl_original_model_path=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}/{unknown_device}/transfer-model/model.h5"
        transfer_model.save(tl_original_model_path)

      ## till here, we have created a model using X number of top features 


      #### Defining Anomaly Error Threshold for the Transferred Model

      tl_results_file_writing('\n\n  <<  Transferred Model: Defining Anomaly Threshold  >>  \n\n')
    
      df_benign_unknown_device_test_predictions = transfer_model.predict(scaled_df_unknown_device_benign_sample_test[unknown_fs])
      unknown_difference = scaled_df_unknown_device_benign_sample_test[unknown_fs] - df_benign_unknown_device_test_predictions
    
      unknown_mse = np.mean(np.power(scaled_df_unknown_device_benign_sample_test[unknown_fs] - df_benign_unknown_device_test_predictions, 2), axis=1) # Mean-Squared-Error is calculated for each row,
      unknown_mean = unknown_mse.mean() # We calculate the mean of the whole series, that is the mean mse for all the benign test cases.
      unknown_std = unknown_mse.std() # And we calculate the standard deviation of the mse series, containing only the test cases.

      unknown_mse = pd.Series(unknown_mse) #converting np array to dataframe series

      tl_results_file_writing(str("MIN of MSE: " + str(unknown_mse.min())) + "\n")
      tl_results_file_writing(str("MAX of MSE: " + str(unknown_mse.max())) + "\n")
      tl_results_file_writing(str("MEAN of MSE: " + str(unknown_mean)) + "\n")
      tl_results_file_writing(str("STD of MSE:  " + str(unknown_std)) + "\n")

      unknown_mse_min = unknown_mse.min()
      unknown_mse_max = unknown_mse.max()
      tl_results_file_writing(str("\n Max Threshold = MSE_MAX: " + str(unknown_mse_max))  + "\n")

      unknown_max_std_multiplier = int((unknown_mse_max - unknown_mean) / unknown_std)
      tl_results_file_writing(str("\n MAX_STD_Multiplier = " + str(unknown_max_std_multiplier)) + "\n")

      transfer_model_error_distribution=f"/content/drive/My Drive/thesis-unsub/unsub/anomaly-transferlearning/looped_run_without_fs/{malware}/{known_device}/{unknown_device}/transfer-model/error-distribution.png"
      unknown_mse_plot = unknown_mse.plot.hist(bins=20, color='#0033cc')
      unknown_mse_plot.set_facecolor('#ffffff')
      unknown_mse_plot.set_xlim((unknown_mse_min, unknown_mse_max))
      unknown_mse_plot.set_ylim((0, unknown_mse[unknown_mse <= unknown_mse_max].count()))
      fig = unknown_mse_plot.get_figure()
      fig.savefig(transfer_model_error_distribution)


      ### Getting a base accuracy of the model using MSE-MAX threshold.
      post_tl_results = unknown_anomaly_detect_func(scaled_df_unknown_device_benign_sample_test, scaled_df_unknown_device_attack, unknown_device, unknown_mse_max, transfer_model, scaler, unknown_fs, unknown_device_confusion_matrix_figure_transfer_model_threshold)
      post_tl_base_acc = post_tl_results[0]    
      post_tl_anomaly_detect_01_X_test = post_tl_results[1]
      post_tl_anomaly_detect_01_Y_test  = post_tl_results[2]


      ### Finding the threshold with best scores

      tl_results_file_writing('\n\n <<--=== Finding Best Performning Threshold After Transfer Learning ===-->> \n')

      unknown_quantile_mse=unknown_mse

      transfer_model_best_results = tl_best_results_finder(post_tl_base_acc, unknown_mse, unknown_quantile_mse, transfer_model, scaler, malware, unknown_device, post_tl_anomaly_detect_01_X_test, post_tl_anomaly_detect_01_Y_test, unknown_device_confusion_matrix_figure_transfer_model_best_percentile)
      transfer_model_best_threshold = transfer_model_best_results[0]
      transfer_model_best_percentile = transfer_model_best_results[1]


      #### FINALLY, finding the performance of the Transfer Learning model on the unknown data

      final_performance_results = unknown_anomaly_detect_func(scaled_df_unknown_device_benign_remaining, scaled_df_unknown_device_attack, unknown_device, transfer_model_best_threshold, transfer_model, scaler, unknown_fs, unknown_device_confusion_matrix_figure_transfer_model_final_performance)
      tl_model_best_accuracy = final_performance_results[0]
      tl_model_best_precision = final_performance_results[3]
      tl_model_best_recall = final_performance_results[4]
      tl_model_best_f1score = final_performance_results[5]

      results_file_writing(str(unknown_device))
      results_file_writing('\n')
      results_file_writing(str('ACCURACY: ' + str(tl_model_best_accuracy)))
      results_file_writing('\n')
      results_file_writing(str('PRECISION: ' + str(tl_model_best_precision)))
      results_file_writing('\n')
      results_file_writing(str('RECALL: ' + str(tl_model_best_recall)))
      results_file_writing('\n')
      results_file_writing(str('F1-SCORE: ' + str(tl_model_best_f1score)))
      results_file_writing('\n ------- \n')      



